#  <span style="color:orange">Reglas de Asociacion en </span>

Fuente: https://pycaret.org/association-rules/

# 1.0 Objetivo del tutorial
Bienvenido al Tutorial de minería de reglas de asociación. Este tutorial asume que usted es nuevo en PyCaret y busca comenzar con la minería de reglas de asociación usando el módulo `pycaret.arules`.

En este tutorial aprenderemos:

* **Obtencion de datos:** ¿Cómo importar datos desde el repositorio de PyCaret?
* **Configuración del entorno:** ¿Cómo configurar el experimento en PyCaret para comenzar con la minería de reglas de asociación?
* **Crear modelo:** ¿Cómo crear un modelo para evaluar resultados?
* **Plot Model:** ¿Cómo analizar el modelo usando varios gráficos?


# 2.0 ¿Qué es la minería de reglas de asociación?
El aprendizaje de reglas de asociación es un método de aprendizaje automático basado en reglas para descubrir relaciones entre variables en grandes bases de datos. 
Su objetivo es identificar reglas opatrones en bases de datos utilizando algunas medidas de interés. Por ejemplo, la regla {cebollas, patatas} -> {hamburguesa} que se encuentra en los datos de ventas de un supermercado indicaría que si un cliente compra cebollas y patatas juntas, es probable que también compre hamburguesas. Dicha información se puede utilizar como base para decisiones sobre actividades de marketing como, por ejemplo, precios promocionales o ubicaciones de productos.


# 3.0 Descripción general del módulo de reglas de asociación en PyCaret

El módulo de reglas de asociación de PyCaret (`pycaret.arules`) es un módulo de aprendizaje automático supervisado que se utiliza para descubrir relaciones entre variables un conjunto de datos. Este módulo transforma automáticamente cualquier base de datos transaccional en una forma que sea aceptable para el algoritmo a priori. **A priori** es un algoritmo para la extracción frecuente de conjuntos de elementos y el aprendizaje de reglas de asociación sobre bases de datos relacionales.

# 4.0 Dataset 

Para este tutorial usaremos una pequeña muestra del conjunto de datos de UCI llamada **Conjunto de datos de retail**. Este es un conjunto de datos transaccionales que contiene transacciones que ocurren entre el 01/12/2010 y el 09/12/2011 para una tienda minorista en línea registrada y con sede en el Reino Unido. La empresa vende principalmente regalos únicos para todas las ocasiones. Muchos clientes de la empresa son mayoristas. La descripción breve de los atributos es la siguiente:

- **InvoiceNo:** Número de factura. Nominal, un número integral de 6 dígitos asignado de forma única a cada transacción. Si este código comienza con la letra 'c', indica una cancelación.
- **StockCode:** Código de producto (artículo). Nominal, un número integral de 5 dígitos asignado de forma única a cada producto distinto.
- **Description:** Nombre del producto (artículo). Nominal.
- **Cantidad:** Las cantidades de cada producto (artículo) por transacción. Numérico.
- **InvoiceData:** Fecha y hora de entrada. Numérico, el día y la hora en que se generó cada transacción.
- **Precio unitario:** Precio unitario. Numérico, Precio del producto por unidad en libras esterlinas.
- **CustomerID:** Número de cliente. Nominal, un número integral de 5 dígitos asignado de forma exclusiva a cada cliente.
- **País:** Nombre del país. Nominal, el nombre del país donde reside cada cliente.

#### Reconocimiento del conjunto de datos:
Dr. Daqing Chen, Director: Grupo de análisis público. chend@lsbu.ac.uk, Escuela de Ingeniería, Universidad London South Bank, Londres SE1 0AA, Reino Unido.


# 5.0 Obtención de los Datos

In [1]:
from pycaret.datasets import get_data
data = get_data('france')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,12/1/2010 8:45,3.75,12583.0,France
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,12/1/2010 8:45,3.75,12583.0,France
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,12/1/2010 8:45,3.75,12583.0,France
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,12/1/2010 8:45,0.85,12583.0,France
4,536370,21883,STARS GIFT TAPE,24,12/1/2010 8:45,0.65,12583.0,France


**Note:** If you are downloading the data from original source, you have to filter `Country` for 'France' only, if you wish to reproduce the results in this experiment.

In [2]:
#check the shape of data
data.shape

(8557, 8)

# 6.0 Setting up Environment in PyCaret

`setup()` function initializes the environment in PyCaret and transforms the transactional dataset into a shape that is acceptable to Apriori algorithm. It requires three mandatory parameters: pandas dataframe, `transaction_id` which is the name of column representing transaction id and will be used to pivot the matrix; and `item_id` which is the name of the column used for creation of rules. Normally, this will be the variable of interest. You can also pass an optional parameter `ignore_items` to ignore certain values for creation of rule.

In [3]:
from pycaret.arules import *

In [4]:
exp_arul101 = setup(data = data, 
                    transaction_id = 'InvoiceNo',
                    item_id = 'Description') 

Description,Value
session_id,4761
# Transactions,461
# Items,1565
Ignore Items,None


Una vez que la configuración se ejecuta con éxito, imprime la tabla con:

- **# Transacciones:** Número único de transacciones en el conjunto de datos. En este caso, un ʻInvoiceNo` único. <br/>
<br/>
- **# elementos:** número único de elementos en el conjunto de datos. En este caso, "Descripción". <br/>
<br/>
- **Ignorar elementos:** Elementos que se deben ignorar en la minería de reglas. Muchas veces hay relaciones que son demasiado obvias y es posible que desee ignorarlas para este análisis. Por ejemplo: muchos conjuntos de datos transaccionales contendrán el costo de envío, que es una relación muy obvia que se puede ignorar en `setup ()` usando el parámetro ʻignore_items`.

# 7.0 Creación del Modelo

Crear un modelo de reglas de asociación es simple. `create_model ()` no requiere parámetros obligatorios. Tiene 4 parámetros opcionales que son los siguientes:

- **métrica:** métrica para evaluar si una regla es de interés. El valor predeterminado es 'confianza'. Otras métricas disponibles incluyen 'support', 'lift', 'leverage', 'conviction' ('soporte', 'impulso', 'apalancamiento', 'convicción'). <br/>
<br/>
- **umbral:** umbral mínimo para la métrica de evaluación, a través del parámetro `metric`, para decidir si una regla candidata es de interés. El valor predeterminado es "0.5". <br/>
<br/>
- **min_support:** Un valor entre 0 y 1 para la compatibilidad mínima de los conjuntos de elementos devueltos. El soporte se calcula como la fracción `transacciones_donde_elemento (s) _ocurren / total_transacciones`. El valor predeterminado es "0.05". <br/>
<br/>
- **redondeo:** El número de métricas de decimales en la cuadrícula de puntuación se redondeará a. <br/>


In [5]:
model1 = create_model() #model created and stored in model1 variable.

In [6]:
print(model1.shape) #141 rules created.

(141, 9)


In [7]:
model1.head() #see the rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(JUMBO BAG WOODLAND ANIMALS),(POSTAGE),0.0651,0.6746,0.0651,1.0000,1.4823,0.0212,inf
1,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.0868,0.1085,0.0846,0.9750,8.9895,0.0752,35.6616
2,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.0868,0.1171,0.0846,0.9750,8.3236,0.0744,35.3145
3,"(POSTAGE, SET/20 RED RETROSPOT PAPER NAPKINS ,...",(SET/6 RED SPOTTY PAPER PLATES),0.0716,0.1085,0.0694,0.9697,8.9406,0.0617,29.4208
4,"(POSTAGE, SET/6 RED SPOTTY PAPER PLATES, SET/2...",(SET/6 RED SPOTTY PAPER CUPS),0.0716,0.1171,0.0694,0.9697,8.2783,0.0610,29.1345


# 8.0 Setup con `ignore_items`

En `model1` creado anteriormente, observe que la regla número 1 es de` JUMBO BAG WOODLAND ANIMALS` con `POSTAGE` que es muy obvio. En el siguiente ejemplo, usaremos el parámetro ʻignore_items` en setup ()` para ignorar `POSTAGE` del conjunto de datos y volver a crear el modelo de reglas de asociación.

In [8]:
exp_arul101 = setup(data = data, 
                    transaction_id = 'InvoiceNo',
                    item_id = 'Description',
                    ignore_items = ['POSTAGE']) 

Description,Value
session_id,3769
# Transactions,461
# Items,1565
Ignore Items,['POSTAGE']


In [19]:
model2 = create_model(metric='lift')

In [20]:
print(model2.shape) #notice how only 45 rules are created vs. 141 above.

(52, 9)


In [21]:
model2.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(CHILDRENS CUTLERY DOLLY GIRL ),(CHILDRENS CUTLERY SPACEBOY ),0.0629,0.0586,0.0542,0.8621,14.7190,0.0505,6.8254
1,(CHILDRENS CUTLERY SPACEBOY ),(CHILDRENS CUTLERY DOLLY GIRL ),0.0586,0.0629,0.0542,0.9259,14.7190,0.0505,12.6508
2,"(ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...",(ALARM CLOCK BAKELIKE RED ),0.0629,0.0803,0.0542,0.8621,10.7409,0.0492,6.6681
3,(ALARM CLOCK BAKELIKE RED ),"(ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...",0.0803,0.0629,0.0542,0.6757,10.7409,0.0492,2.8894
4,"(ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...",(ALARM CLOCK BAKELIKE GREEN),0.0629,0.0846,0.0542,0.8621,10.1901,0.0489,6.6367


# 9.0 Visualizar el Modelo

In [22]:
plot_model(model2)

In [23]:
plot_model(model2, plot = '3d')